<a href="https://colab.research.google.com/github/RenaAbbasova/Tesla_stock_prediction/blob/main/Tesla_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Problem Statement and Agenda:

One of the applications of Time Series Forecasting is to predict opening stock prices, closing stock prices, and the volume of stocks to be traded, among others. In this dataset, we aim to forecast the future behavior of the stock market, focusing on the average stock price data of Tesla spanning from 2010 to 2022.

For this case study, we will divide our data into training and test sets, build our models on the training data, forecast for the test data timestamps, and then evaluate using the Root Mean Squared Error (RMSE) model evaluation metric.

The following topics will be covered in this case study:

Exploratory Data Analysis
ARIMA/SARIMA models (with and without exogenous variables)
Facebook Prophet Model
LSTM Model (Deep Learning Model)


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
sns.set()
%matplotlib inline
from pandas import Series
from numpy import log
import plotly.express as px # high level interface
import plotly.graph_objects as go # lower level interface

In [ ]:
# Import libraries for advanced modeling
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# read the data
data=pd.read_csv('/content/TSLA.csv')

In [ ]:
data

In [ ]:
# Analyze moving averages to capture trends
data['MA50'] = data['Close'].rolling(window=50).mean()
data['MA200'] = data['Close'].rolling(window=200).mean()
data[['Close', 'MA50', 'MA200']].plot(figsize=(14, 7), title="Tesla Closing Price with Moving Averages")
plt.show()

In [ ]:
data1=data.copy()

In [ ]:
data1.set_index('Date',inplace=True)

In [ ]:
data1.head()

In [ ]:
df = data.copy()

#EDA

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
# Calculate the Typical Price
df['Typical_Price'] = (df['High'] + df['Low'] + df['Close']) / 3

# Calculate TPV (Typical Price * Volume)
df['TPV'] = df['Typical_Price'] * df['Volume']

# Calculate Cumulative TPV and Cumulative Volume
df['Cumulative_TPV'] = df['TPV'].cumsum()
df['Cumulative_Volume'] = df['Volume'].cumsum()

# Calculate VWAP
df['VWAP'] = df['Cumulative_TPV'] / df['Cumulative_Volume']



In [ ]:
df.columns

In [ ]:

# we going to keep only 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP' variables

# Drop intermediate columns if not needed
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']]

In [ ]:
df.head()

# DURBIN - WATSON TEST

In [ ]:
 #Durbin-Watson test going to be applied to residuals from time series models to diagnose autocorrelation.

In [ ]:
import statsmodels.api as sm
print(sm.stats.durbin_watson(df['VWAP']))
print(sm.stats.durbin_watson(df['Open']))
print(sm.stats.durbin_watson(df['High']))
print(sm.stats.durbin_watson(df['Low']))
print(sm.stats.durbin_watson(df['Close']))
print(sm.stats.durbin_watson(df['Volume']))

In [ ]:

# The presence of strong positive autocorrelation in the residuals of the 'VWAP', 'Open', 'High', 'Low',
# and 'Close' variables implies that their current values are highly dependent on their past values.

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
start_date = df['Date'].min()
end_date = df['Date'].max()
print(start_date)
print(end_date)

In [ ]:
df.shape

In [ ]:
# check frequency of VWAP variable
df.VWAP.plot(figsize=(20,5),title='VWAP')
plt.show()

In [ ]:
# Convert 'date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df ['year'] = df['Date'].dt.year
df['month'] = df ['Date'].dt.month


In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x=df['year'], y =df['VWAP'],palette='pastel')
plt.grid()

In [ ]:
# We can observe from the graph how the volume-weighted average price has been increasing since 2010,
# with the best year for selling being 2021.

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x=df['month'], y =df['VWAP'],palette='pastel')
plt.grid()

In [ ]:
# Selling in November was good, as the prices were high.


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose


In [ ]:
# Decompose the time series into trend, seasonal, cyclical, and irregular components.
decompose = seasonal_decompose(df['VWAP'], model='additive', period=365)

# Extract the components
trend = decompose.trend
seasonal = decompose.seasonal
cyclical = df['VWAP']  - trend - seasonal
irregular = decompose.resid

In [ ]:
# Plot the components
plt.subplot(411)
plt.plot(df['VWAP'], label='Original')
plt.legend(loc='upper left')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='upper left')
plt.subplot(413)
plt.plot(seasonal, label='Seasonality')
plt.legend(loc='upper left')
plt.subplot(414)
plt.plot(irregular, label='Residuals')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

# Q-Q plot

In [ ]:
# Let´s see whether a dataset follow a normal distribution

In [ ]:
import scipy.stats
import pylab

In [ ]:
scipy.stats.probplot(df.VWAP, plot=pylab)
plt.title("QQ plot for Volume Waighted Price Average")
pylab.show()

In [ ]:
import scipy.stats as stats

# Replace 'desired_distribution' with the name of the distribution you want to test against (e.g., 'expon', 'gamma', 'uniform', etc.)
stats.probplot(df['VWAP'], dist='expon', plot=plt)
plt.title("QQ plot for Volume Weighted Price Average")
plt.show()


In [ ]:
trend=decompose.trend
seasonality=decompose.seasonal
residual=decompose.resid

print('Trend', '\n', trend.head(12), '\n')

print('Seasonality', '\n', seasonality.head(12), '\n')

print('Residual', '\n', residual.head(12), '\n')

In [ ]:
# Creating new column
df['Time_Stamp']=pd.DataFrame(df, columns=['Date'])
df['Time_Stamp'] = pd.to_datetime(df['Time_Stamp'])
# set index column 'Time_Stamp'
df_final = df.set_index('Time_Stamp')


In [ ]:
df_final.head()

# Spit the data into training and test before building Time Series Forecasting

In [ ]:
df_final.index.max()

In [ ]:
df_final.index.min()

In [ ]:
# split into training and tesing data
train_df = df_final[pd.to_datetime(df_final['Date']) < pd.to_datetime('2017-10-04')]

test_df = df_final[pd.to_datetime(df_final['Date']) >= pd.to_datetime('2017-10-04')]

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_final = train_df[['VWAP']]
test_final = test_df[['VWAP']]

In [ ]:
train_final.head()

In [ ]:
test_final.head()

# Augmented Dickey Fuller Test

In [ ]:
# To check time series stationary or not

from statsmodels.tsa.stattools import adfuller

In [ ]:
adfuller(train_final['VWAP'])

In [ ]:
def check_adftest(timeseries):
  result=adfuller(timeseries)
  print('Augmented Dickey Fuller Test')
  labels = ['ADF test', 'P-value','#Lags', 'No of observation']

  for i, j in zip(result,labels):
    print(j + '------->'+str(i))

  if result[1] <= 0.05:
    print('Strong evidence against Null Hypothesis and my time series is Stationary')

  else:
    print('Weak Evidence against Null Hypothesis and my time series is Non-Stationary')

In [ ]:
check_adftest(train_final['VWAP'])

In [ ]:
# as time series is non - stationary
# we aply Differencing technique

In [ ]:
# Difference the time series and plot
diff_data = train_final['VWAP'].diff().dropna()
plt.plot(diff_data)
plt.title('Differenced Time Series')
plt.show()




In [ ]:
check_adftest(diff_data)

In [ ]:
# This code generates combinations of parameters for a SARIMA (Seasonal Autoregressive Integrated Moving Average) model.
# SARIMA models are extensions of the ARIMA model that include seasonal components.

In [ ]:
import itertools
p = q = range(0,3)

d = range(0,1)

pdq = list(itertools.product(p, d, q)) # Trend

model_pdq = [(x[0], x[1], x[2], 5) for x in list(itertools.product(p,d,q))] # Seasonality

print('Example of combination for model...........')
print('Model : {}{}'.format(pdq[1],model_pdq[1]))
print('Model : {}{}'.format(pdq[0],model_pdq[0]))
print('Model : {}{}'.format(pdq[2],model_pdq[2]))
print('Model : {}{}'.format(pdq[1],model_pdq[2]))

# Build SARIMAX model - Seasonality Autoregressive integrated moving average with external factor


SARIMAX is a statistical model used for time series forecasting. It's an extension of the ARIMA model that incorporates additional features to handle seasonality and exogenous variables (variables external to the model that can influence the time series being analyzed).

In [ ]:
ex_train =  train_df
ex_test = test_df

In [ ]:
ex_train.head()

In [ ]:
df_obj=pd.DataFrame(columns=['param','seasonal','AIC'])
df_obj

In [ ]:
# Ensure train_final and ex_train are numeric and aligned
train_final = train_final.select_dtypes(include=['float64', 'int64'])
ex_train = ex_train.select_dtypes(include=['float64', 'int64'])
ex_train = ex_train.reindex(train_final.index)

# Initialize an empty list to store results
results = []

for param in pdq:
    for param_seasonal in model_pdq:
        try:
            model = sm.tsa.statespace.SARIMAX(train_final, exog=ex_train, order=param, seasonal_order=param_seasonal,
                                              enforce_stationarity=False, enforce_invertibility=False)
            result_SARIMAX = model.fit()
            print('SARIMAX{}{} - AIC:{}'.format(param, param_seasonal, result_SARIMAX.aic))

            # Append the results to the list
            results.append({'param': param, 'seasonal': param_seasonal, 'AIC': result_SARIMAX.aic})
        except Exception as e:
            print(f"Error with SARIMAX{param}{param_seasonal}: {e}")

# Convert the list of dictionaries to a DataFrame
df_obj = pd.DataFrame(results)
print(df_obj)


In [ ]:
df_obj.head()

In [ ]:
best_model = df_obj.loc[df_obj['AIC'].idxmin()]
best_model

In [ ]:
#(0, 0, 0)	(0, 0, 0, 5)		-38738.804187
model = sm.tsa.statespace.SARIMAX(train_final, exog=ex_train, order=(0, 0, 0),
                                   seasonal_order=(0, 0, 0, 5),
                                   enforce_stationarity=False, enforce_invertibility=False)
result = model.fit()
print(result.summary())


#### The SARIMAX model successfully completed, demonstrating a strong fit to the VWAP data with a nearly perfect coefficient for VWAP (1.0000, 𝑝 < 0.001). However, other predictors (Open, High, Low, Close, Volume) showed no significant impact, suggesting they may not contribute meaningfully to the model's performance. The model encountered some warnings regarding singular covariance and convergence issues, which may indicate potential instability in the parameter estimates. Diagnostic tests (Ljung-Box, Jarque-Bera) highlight residual autocorrelation and non-normality. Further refinement, including addressing heteroskedasticity and ensuring parameter stability, could enhance model reliability.

In [ ]:
result.plot_diagnostics(figsize=(16,8))
plt.show()

In [ ]:
# Define the columns used during training
exog_train_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']

# Filter ex_test to match the training exogenous variables
ex_test = ex_test[exog_train_columns]

# Ensure no missing values and proper numeric types
ex_test = ex_test.fillna(0).astype(float)

# Perform forecasting
predict_SARIMAX = result.get_forecast(steps=len(test_df), exog=ex_test)
predicted_mean = predict_SARIMAX.predicted_mean
print(predicted_mean)


In [ ]:
from sklearn.metrics import *
import math
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
rmse = sqrt(mean_squared_error(test_final.VWAP, predict_SARIMAX.predicted_mean, squared=False))
print(rmse)

#### The RMSE (Root Mean Square Error) value of 3.637539544312077e-05 indicates an extremely low error, suggesting that the SARIMAX model is highly accurate in its predictions.

In [ ]:
plt.plot(train_final, label='Training Data')
plt.plot(test_final, label='Test Data')
plt.plot(test_final.index, predict_SARIMAX.predicted_mean, label='predicted Model -SARIMAX')
plt.legend(loc='best')
plt.grid();

# Sarima Model

In [ ]:
model = sm.tsa.statespace.SARIMAX(train_final['VWAP'],order=(1,1,2),
                                   seasonal_order=(1,0,2,5),
                                   enforce_stationarity=False, enforce_invertibility=False)
result = model.fit()
print(result.summary())

In [ ]:
predict_SARIMA = result.get_forecast(steps=len(test_df), exog=ex_test)
predict_SARIMA.predicted_mean

In [ ]:
rmse = sqrt(mean_squared_error(test_final.VWAP, predict_SARIMA.predicted_mean, squared=False))
print(rmse)

In [ ]:
plt.plot(train_final, label='Training Data')
plt.plot(test_final, label='Test Data')
plt.plot(test_final.index, predict_SARIMAX.predicted_mean, label='predicted Model -SARIMAX')
plt.plot(test_final.index, predict_SARIMA.predicted_mean, label='predicted Model -SARIMA')
plt.legend(loc='best')
plt.grid();

##### The SARIMAX model performs close to the actual values and provides satisfactory forecast accuracy, it suggests that incorporating exogenous variables has helped capture important information in our time series data.

# LSTM Model

In [ ]:
df_final.head()

In [ ]:
### LSTM Model Implementation
# Scale data for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(df[['VWAP']])

In [ ]:
# Split data into train and test
train_size = int(len(df_final) * 0.8)
train, test = df_final[:train_size], df_final[train_size:]

In [ ]:
# Prepare data for LSTM
train_scaled = data_scaled[:train_size]
test_scaled = data_scaled[train_size - 50:]




In [ ]:
# Create sequences for LSTM
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(data[i + sequence_length])
    return np.array(sequences), np.array(targets)

sequence_length = 50
train_sequences, train_targets = create_sequences(train_scaled, sequence_length)
test_sequences, test_targets = create_sequences(test_scaled, sequence_length)

In [ ]:
# Reshape data for LSTM
train_sequences = train_sequences.reshape(train_sequences.shape[0], train_sequences.shape[1], 1)
test_sequences = test_sequences.reshape(test_sequences.shape[0], test_sequences.shape[1], 1)

In [ ]:
# Build the LSTM model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])

In [ ]:
# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
lstm_model.fit(train_sequences, train_targets, epochs=10, batch_size=32, verbose=1)

In [ ]:
# Make predictions
lstm_predictions = lstm_model.predict(test_sequences)

In [ ]:

# Reverse scaling
lstm_predictions = scaler.inverse_transform(lstm_predictions)
test_targets = scaler.inverse_transform(test_targets.reshape(-1, 1))

In [ ]:
# Plot predictions vs actual values
plt.figure(figsize=(14, 7))
plt.plot(test_targets, label='Actual Values', color='blue')
plt.plot(lstm_predictions, label='LSTM Predictions', color='orange')
plt.title('LSTM Predictions vs Actual Values')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.legend()
plt.show()

In [ ]:
# Calculate RMSE
lstm_rmse = sqrt(mean_squared_error(test_targets, lstm_predictions))
print("LSTM RMSE:", lstm_rmse)

## **Conclusion**

In this project, we focused on predicting the VWAP using 12 year's worth of time series data. Throughout the project, we followed a systematic approach to build and evaluate time series models.

We began by reading and preprocessing the data, ensuring that it was in a suitable format for analysis. We then conducted exploratory data analysis (EDA) to gain insights into the patterns, trends, and seasonality present in the data.

To proceed with modeling, we first checked the stationarity of the time series. Since the data was non - stationary, we performed differencing technique.

We applied Durbin-Watson test to residuals from time series models to diagnose autocorrelation.

We built  models like Sarima, Sarimax and LSTM model. The performance of these models was evaluated based on both the p-value and root mean squared error (RMSE).

After evaluating the models, we found that Sarimax model outperformed.


